# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik


The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for vectors work in approximating semantic similarity of phrases when compared to human judgements.

This lab uses code from `dist_erk.py` which contains functions similar to those shown in the lecture. You can use either functions to solve these tasks.

In [6]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

**Important**: All necessary files which are used in this notebook are available on mlt-gpu, check `/srv/data/computational-semantics-assignment-02`.

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus (`wikipedia.txt`. This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/).

When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [8]:
corpus_dir = '/srv/data/computational-semantics-assignment-02/'
#corpus_dir = "computational-semantics-assignment-02/"

In [43]:
# In case we want to test this on a smaller corpus:
!mkdir small_data
!shuf -n 100000 /srv/data/computational-semantics-assignment-02/wikipedia.txt > small_data/small_wiki.txt

mkdir: small_data: File exists
zsh:1: command not found: shuf


In [44]:
small_dir = './small_data/'

Comments:

Since the dataset requires a long time to be processed, a small selection will be used instead.
In the next section the Pickle module will be used to binarize otherwise big results.

Notice that "small_dir" will be used instead of the directory containing "wikipedia.txt".

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [28]:
numdims = 1000
svddim = 5

# which words to use as targets and context words?
# we need to count the words and keep only the N most frequent ones
# which function would you use here with which variable?
ktw = do_word_count(small_dir,numdims)

wi = make_word_index(ktw)
words_in_order = sorted(wi.keys(), key=lambda w:wi[w])
# create different spaces (the original matrix space, the ppmi space, the svd space)
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(small_dir, wi, numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(space_1k, numdims,svddim)
print('done.')

create count matrices
ppmi transform
svd transform
done.


In [46]:
# pickle data:

!mkdir pickles
import pickle
def picklesave(data, name):
    filename = 'pickles/'+name+'.pickle'
    with open(filename, "ab") as outfile:
        pickle.dump(data, outfile)

picklesave(ktw, 'ktw_1k')
picklesave(space_1k, 'space_1k')
picklesave(ppmispace_1k, 'ppmispace_1k')
picklesave(svdspace_1k, 'svdspace_1k')

mkdir: pickles: File exists


Comment:
Notice that the data is store in pickle format to save resources.

In [47]:
# to import pickled data:

with open('pickles/ktw_1k.pickle', 'rb') as file:
    ktw = pickle.load(file)
with open('pickles/space_1k.pickle', 'rb') as file:
    space_1k = pickle.load(file)
with open('pickles/ppmispace_1k.pickle', 'rb') as file:
   ppmispace_1k = pickle.load(file)
with open('pickles/svdspace_1k.pickle', 'rb') as file:    
    svdspace_1k = pickle.load(file)

In [56]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])

house: [41 43 25  8 15  5 11  6  4  1  0  0  2  1  2  2  4  0  2  2  2  0  0  1
  0  0  0  2  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0
  0  1  0  0  0  0  1  2  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  2  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  2  0  0  0  1  0
  0  1  0  0  0  0  0  0  0  3  0  0  0  0  0  0  1  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  1  0  

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. All matrices are available on mlt-gpu: `ktw_wikipediaktw.npy`, `raw_wikipediaktw.npy`, `ppmi_wikipediaktw.npy`, `svd50_wikipedia10k.npy`. Make sure they are in your path, because they will be loaded below.

In [9]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load(corpus_dir+'ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load(corpus_dir+'raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load(corpus_dir+'ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load(corpus_dir+'svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [10]:
# testing semantic space
print('house:', space_10k['house'])

house: [2554 3774 3105 ...    0    0    0]


Comments:

Both, 1k vectors and 10k vectors, were correctly visualized.

## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [11]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw_10k))))
print('number of available word pairs to test:', len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 155
number of available word pairs to test: 774


Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [12]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))

Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [13]:
raw_similarities  = [cosine(w1, w2, space_10k) for w1, w2 in word_pairs]
ppmi_similarities = [cosine(w1, w2, ppmispace_10k) for w1, w2 in word_pairs]
svd_similarities  = [cosine(w1, w2, svdspace_10k) for w1, w2 in word_pairs]

Comments:

From the "dist_erk.py" file, the cosine function is used to compare words from different spaces in a very convenient way by allowing the "space" argument.
In the following code block a comparison is given as a means for quick visualization.

In [14]:
# The following is a quick visualization for some pairs of words across different spaces.

pairs = [(w1, w2) for w1, w2 in word_pairs]
for pair, raw_similary, ppmi_similarity, svd_similarity in zip(pairs[:20], raw_similarities[:20], ppmi_similarities[:20], svd_similarities[:20]):
    print(f" Pair: {pair}\t\t Raw: {100*raw_similary:9.2f}\t PPMI: {100*ppmi_similarity:6.2f}\t SVD: {100*svd_similarity:8.2f}")


 Pair: ('stick', 'sword')		 Raw:     82.25	 PPMI:   7.67	 SVD:    73.60
 Pair: ('cabin', 'cabinet')		 Raw:     88.72	 PPMI:   3.67	 SVD:    39.83
 Pair: ('chicken', 'sparrow')		 Raw:     81.86	 PPMI:   2.61	 SVD:    37.22
 Pair: ('bag', 'gate')		 Raw:     86.41	 PPMI:   6.01	 SVD:    59.76
 Pair: ('bull', 'trumpet')		 Raw:     92.09	 PPMI:   3.01	 SVD:    36.90
 Pair: ('balloon', 'brick')		 Raw:     83.66	 PPMI:   5.53	 SVD:    58.93
 Pair: ('helicopter', 'jet')		 Raw:     91.06	 PPMI:  14.65	 SVD:    91.94
 Pair: ('cape', 'spider')		 Raw:     83.96	 PPMI:   4.53	 SVD:    49.44
 Pair: ('boots', 'mouse')		 Raw:     75.89	 PPMI:   4.95	 SVD:    55.08
 Pair: ('kite', 'marble')		 Raw:     87.30	 PPMI:   3.43	 SVD:    44.88
 Pair: ('box', 'horse')		 Raw:     91.96	 PPMI:   8.51	 SVD:    74.80
 Pair: ('doll', 'elephant')		 Raw:     78.23	 PPMI:   5.28	 SVD:    79.26
 Pair: ('brick', 'cannon')		 Raw:     93.86	 PPMI:   9.03	 SVD:    67.66
 Pair: ('ruler', 'turkey')		 Raw:     67.13	 PPMI:  11

Comments:

It can be easily observed how the three models present conflicting results: comparissons such as "helicopter" and "jet" score highly in all three, but "doll" and "elephant" score lowly in PPMI and highly in the other two models, others such as "book" and "submarine" score highly in the raw space similarity, but lowly in the rest.

Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [15]:
rho, pval = stats.spearmanr(raw_similarities, semantic_similarity)
print("""Raw Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))
print("\n")


rho, pval = stats.spearmanr(ppmi_similarities, semantic_similarity)
print("""PPMI Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))
print("\n")


rho, pval = stats.spearmanr(svd_similarities, semantic_similarity)
print("""SVD Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))
print("\n")


Raw Similarity vs. Semantic Similarity:
rho     = 0.1522
p-value = 0.0000


PPMI Similarity vs. Semantic Similarity:
rho     = 0.4547
p-value = 0.0000


SVD Similarity vs. Semantic Similarity:
rho     = 0.4232
p-value = 0.0000




**Your answer should go here:**

According to the comparison provided above we can conclude that PPMI approximates, by a slight margin, best to human judgements. However, both PPMI and SVD are rather distant from what is expected.
SVD, in its simplicity may lose part of important context, but that loss is, probably, not significant enough (with the dataset that we work with). PPMI presents a good probabilistic approach, which is much better that just co-occurrences in the "raw" similarities.
This reasoning corresponds with the fact that more sophisticated probabilistic approaches work better than just the mere surrounding context of a word.

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [16]:
rho, pval = stats.spearmanr(raw_similarities, visual_similarity)
print("""Raw Similarity vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))
print("\n")

rho, pval = stats.spearmanr(ppmi_similarities, visual_similarity)
print("""PPMI Similarity vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))
print("\n")

rho, pval = stats.spearmanr(svd_similarities, visual_similarity)
print("""SVD Similarity vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))
print("\n")

Raw Similarity vs. Visual Similarity:
rho     = 0.1212
p-value = 0.0007


PPMI Similarity vs. Visual Similarity:
rho     = 0.3838
p-value = 0.0000


SVD Similarity vs. Visual Similarity:
rho     = 0.3097
p-value = 0.0000




**Your answer should go here:**

The results from the visual similarity are consistent with those of the semantic similarity, but in this case PPMI does  even better than SVD and "Raw" similarity does slightly worse.
Judging by the first cosine similarity comparison between Visual Similarity and Semantic Similarity, it being around 0.7 already hints that the results of the operations performed to them, no matter which, should't be too different from each other. This is proved above.


## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions.

For example, we can perform `king - man` and add the resulting vector to `woman` and we hope to get the vector for `queen`. Also, what would be the result of `stockholm - sweden + denmark`? Why? **[3 marks]**

If you want to learn more about vector differences between words (and words in analogy relations), check [this paper](https://aclanthology.org/P16-1158.pdf).

**Your answer should go here:**

When we compute "king - man + woman = queen" we are retaining the unique characteristics of the first semantic element and applying them to the third semantic element. In this case we could say the chracteristics of that first element are equivalent to "royalty"(N) or "regal"(ADJ) and, since those are applied to a person (man), altering the "polarity" of such  element gives us the opposite gender of the "regal entity", that is "queen".
If the same process is applied to "stockholm - sweden + denmark" we are left with the characteristic "city", a subset of the entities that make up a "country". Is interesting how we, then, automatically take into account the most salient relation of "Stockholm", a city, to the following element "Sweden", a country, which is "capital". Thus, we are left with the characteristics "city" and "capital" to "Denmark", which result in no other entity than "Copenhagen".
In the first example there is a relation inseparable from polarity: there's a King and a Queen, and because of this, the relation might be much more evident or may come to mind much more easily than in the second example.

Here is some helpful code that allows us to calculate such comparisons.

In [17]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [18]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])

print(find_similar_to(light - (heavy - long), svdspace_10k)[:10])
print(find_similar_to((heavy - light) + short, svdspace_10k)[:10])
print(find_similar_to((heavy + long) - short, svdspace_10k)[:10])
print(find_similar_to((heavy - short) + long, svdspace_10k)[:10])


[('long', 0.8733111261346901), ('above', 0.8259671977311955), ('around', 0.8030776291120685), ('sun', 0.7692439111243973), ('just', 0.7678481974778111), ('wide', 0.767257431992253), ('each', 0.7665960260861158), ('circle', 0.7647746702909336), ('length', 0.7601066921319761), ('almost', 0.7542351860536628)]
[('short', 0.8352373661313067), ('lengthy', 0.7727685249956753), ('occasional', 0.7581408627401973), ('brief', 0.7532022457352894), ('opening', 0.7125803524384261), ('subsequent', 0.7062952429131073), ('previous', 0.7037420867207232), ('earlier', 0.7013170065105806), ('dramatic', 0.693981584503108), ('included', 0.679120407810342)]
[('heavy', 0.859207118765945), ('tank', 0.7378450414660778), ('fire', 0.7359730012834718), ('gun', 0.7301315160311267), ('ground', 0.725716751109639), ('air', 0.7226095580472032), ('light', 0.7147705210063771), ('combat', 0.7028027559677159), ('shells', 0.7026242122106345), ('landing', 0.698614491946687)]
[('heavy', 0.859207118765945), ('tank', 0.737845041

**Your answer should go here:**



By testing a pair of analogies we can confirm that categories that may come easily to our mind are not computed in a similar way by vector operations. Relations such as "heavy and light" or "long and short" are not captured very precisely.

Looking at a computations such as "light - (heavy - long)", the binary relation should jump to mind very easily with "short" as the answer. Let's think about the characteristics of the semantics elements:
"light" implies "poor weight", "heavy" implies "great weight" and "long" implies "great length", subtracting "great weight" from "great length" does not result in any precise meaning, but leaves us with an equivalence rule: there are two implicit entities that entail polarity, "weight" and "length". Their value, great or poor, becomes relevant after taking into account the third element "light", which reveals "poor weight" to the polarity equivalence. That is probably why "poor length" comes to mind almost inmediately, it is the X missing in the correlation.

All this, however, can only be done when the pure semantics of the element is understood. A distributional approach is, by definition, looking outside the element, and not inside.

The results above capture one of the words themselves, synonyms or words that may describe an object or entity with the characteristics used in the correlations. As discussed previously, this is a result of the observation of the word environment (most likely).


Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [19]:
analogies = [('go', 'went', 'see', 'saw'), 
             ('sea', 'water', 'desert', 'sand'), 
             ('foot', 'feet', 'table', 'tables'), 
             ('love', 'hate', 'night', 'day'), 
             ('man', 'programmer', 'woman', 'programmer')
            ]

for analogy in analogies:
    a,b,c,d = analogy
    print('"{}" is to "{}" as "{}" is to...'.format(a, b, c))
    print(find_similar_to(normalize(svdspace_10k[a]) - (normalize(svdspace_10k[b]) - normalize(svdspace_10k[c])), svdspace_10k)[:10])
    if d not in svdspace_10k:
        print(d, 'not in vector space')

"go" is to "went" as "see" is to...
[('say', 0.7628181643929222), ('find', 0.7437847252209391), ('compare', 0.7431958450277302), ('follow', 0.7417741118308697), ('see', 0.7388715813993888), ('understand', 0.7326518065260651), ('make', 0.7312414374627294), ('add', 0.7250016596149815), ('go', 0.7210654110531284), ('know', 0.7169942587586754)]
"sea" is to "water" as "desert" is to...
[('desert', 0.8631367536147341), ('mediterranean', 0.8065760895275305), ('sea', 0.805695219711723), ('mountains', 0.790307080574741), ('coastal', 0.7892410225619448), ('arctic', 0.7867701788482583), ('ocean', 0.7834812454928333), ('gulf', 0.7754828475869852), ('seas', 0.7748867641663814), ('plains', 0.7722229522760461)]
"foot" is to "feet" as "table" is to...
[('table', 0.9173713911491298), ('ring', 0.8314758899238498), ('anchor', 0.8217527498343041), ('pool', 0.8143377247211394), ('chain', 0.8109512852776265), ('floating', 0.8092960776279715), ('window', 0.8089380556210312), ('box', 0.8041957043663301), ('pi

Comments:

Analogies of the type described earlied ('sea' to 'water' as 'desert' to 'sand') are, as already discussed, not captured in the vectors, but the same goes for grammatical or paradigmatic relations: ('foot', 'feet', 'table', 'tables') or ('go', 'went', 'see', 'saw'), singular-plural or verb paradigms are not taken into account. 
Other relations such as ('man', 'programmer', 'woman', 'programmer'), not as cleary defined as "king and queen", also fail our expectations.
From our set, the only analogy that approaches our understanding is ('love', 'hate', 'night', 'day'), with the relation "night" being "night", the same word, in first place (which is itself a rather odd thing to suggest), but "day" as second. This is a comparatively good result, but we may venture that the result is a mere coincidence: the statistics may come from expressions such as "the difference is like night and day" being frequent in the language. 
This particular case shows that context may be useful to get our desired interpretation of a meaning, but, again, as seen in multiple occasions through this exercise, it is void of the specific semantics pertaining to that word. 


## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset `mitchell_lapata_acl08.txt` is included with this notebook (we also used it in the class).

---

Explanation of the task from Discord channel:

**What are we trying to achieve?**  
We want to create models, which can automatically capture differences between meaning of different phrases. These models should also be as good as we are (humans) in this task. Check example (1) from Mitchell/Lapata paper, it has two sentences which share the same words, but their meaning is completely different. We as humans can clearly see this difference, but how could a machine capture it?

It is intuitive that in order to get a meaning of a phrase, we might combine meaning of individual words in this phrase somehow, but how? First, we represent each word with the frequency vector based on the semantic space that we built before (e.g, frequency space, ppmi space, svd space). In other words, each word's meaning is represented by the number of times other words occur in the context, defined by window size. Now, we have a vector for `discussion` and a vector for `thrive`.

_How do we combine these vectors to get a single vector for the phrase `discussion thrive`?_

Such methods of combining meaning vectors into a single item are called _semantic composition methods_ (literally, because we compose semantic meaning of the phrase from its individuals). During the lecture, we tried different semantic composition methods: additive, multiplicative, combined.

Let's say we multiplied the vectors (went with the multiplicative method) and now we have one vector for our phrase.

Remember, we want to have a model that captures differences between the phrases; it means that if `discussion thrive` is our reference phrase, we need to have a different phrase (high or low similarity phrase) to compare it against the reference one. How do we get this other phrase? Well, this other phrase can be either very similar to the reference or not similar at all, right? Let's say we decided to go with the second option and made/constructed a phrase `discussion digress`, which we know is very dissimilar to the reference phrase. We label this pair of phrases as having a low similarity, e.g., `low` in `hilo`. We can also create a different phrase (e.g., `discussion develop`) and use it as a high similarity phrase when paired with our reference phrase, right? This then would be labeled as `high` in `hilo`. This is what `hilo` in the dataset stands for: known information about how similar the reference phrase and the landmark phrase are.

Now, our main task is to automatically learn the similarities/differences between our reference and our landmark, right? We take the first pair: `discussion thrive` vs. `discussion digress`. We have a vector representation for each of these phrases.

**How do we compare two vectors?**

We use cosine similarity to calculate a single score that would tell us about the similarity between these two vectors. The bigger the cosine, the more similar two vectors are. Cosine ranges from 0 to 1 (0 is very low similarity, 1 is very high similarity). Let's say, we get a cosine of 0.89, it means that according to the multiplicative model (remember, we decided to use multiplicative semantic composition method), these phrases are very similar (cosine is quite high). But wait a second, we know that these two phrases should be of low similarity, right? Because this is what the value in `hilo` tells us about this pair - `discussion thrive` and `discussion digress` are not similar to each other. Clearly, our multiplicative method fails to capture it.

**What can we do to improve our model?**

We can try a different composition method to get a phrase vector from phrase's words: let's replace multiplication with addition. Or we can also use combined method. Let's say we used combined method and run cosine again; this time it tells that the cosine score is 0.45. Ok, this seems to be quite low, and it also agrees with our knowledge that these phrases are indeed not similar.

---

In other words, we need to evaluate different composition models (additive, multiplicative, combined) and analyse how well they perform. **How do we analyse their performance?** Because we have the ground-truth for comparison (`hilo` values), we know whether our phrases are actually similar or not. We want our cosine score to reflect this knowledge: if the score is high, but the groun-truth hilo is low, then we have a problem in the model - it did not learn things well, we need to replace the composition function.

---

_Long story short_: `hilo` is something that we compare our cosine to. `hilo` contains correct answers about similarity between phrases, and cosine should agree with this. If reference-landmark pair are `high` in `hilo`, then cosine should be high enough to reflect that. If cosine is not high in this case, then we look at our model and change the composition function. We need to find the function, which give us cosines that are super close to the ground-truth known `hilo` values.

<img src="res.png" alt="drawing" width="500"/>

Next, we want to compute do we compute correlation between our model's predictions and the ground-truth, something similar to the results from [2] (image above):

---

In `High` and `Low` columns we have mean cosine values.
These are calculated by averaging cosine scores for all pairs of phrases per model.
Rows introduce different models: `add` is additive, `multiply` is multiplicative, etc. (these are all described in the paper). `NonComp` is a baseline model, the most "stupid" one, it should be the worst. `UpperBound` is how humans performed in this task (they were asked to rate similarity between pairs of phrases, 1 is the lowest, 7 is the highest). Why these numbers are in a different scale, not from 0 to 1 like cosine, but from 1 to 7? Because they are not normalised, and authors explicitly said that they are interested in relative differences.

We need models which are closer to human ratings. `Add` model has 0.59 mean for `High` and 0.59 for `Low`, so it did not learn to differentiate between high similarity pairs and low similarity ones. This is a bad model then, we need a better one. `WeightAdd` seems to be doing better, the difference between `High` and `Low` is now 0.01, but it's also quite bad - the difference is not that obvious. The best models are `Multiply` and `Combined`, because their mean cosines for `High` and `Low` are quite different from each other. We can see that these two models gave higher cosines for high similarity pairs (0.42 and 0.38), while giving lower cosines for low similarity pairs (0.28 and 0.28). And this is a good result - it shows that these two composition functions are so far the best in (i) giving high cosine go highly similar pairs and low cosine to very dissimilar pairs, and (ii) keeping the distance (range) between high and low cosines quite large.

**However, we can't say how far/close they are when compared to human performance (UpperBound) since human scores are not normalised.**

Still, we want to choose a model which is the closest to humans. This is why we want to run **the correlation test.**

How do we perform the correlation test? We need to see how well *each model* correlates with human judgements. So for each model, we would have a vector of cosine values this model gives for each pair that we have. For example, let's say we have three pairs of phrases and our cosine values from additive model are the following ones: `[0.89, 0.40, 0.70]`. Now we need to get a vector of the same size, but for human scores. What do we have for human scores? We have multiple participants, which means that a single phrase can be evaluated by multiple participants. Let's say, the first phrase has scores from two participants (`6, 7`), so what we would do is that we would average it to have a single number (`(6 + 7) / 2 = 6.5`). With this, we can get a mean vector of human scores per item: `[6.5, 3, 6]`.

Now, we have two vectors and we can run Spearman correlation on these vectors. This is what exactly what the third column in the Results table is showing (the correlation value). There is also a p-value (denoted right below the table).

**What is your ultimate task in this part of the assignment?**

(i) Process the dataset and extract `reference - landmark` pairs; you can use the code from the lecture as something to start with. Try to keep information about human rating (`input`) and high/low similarity (`hilo`), because you will need it for correlation tests. Also, you might want to keep the information about participant id (will be useful for getting average numbers for correlation tests). Which format you should use to keep all this data? It's up to you, but a dictionary-like format could be a good choice.

(ii) Build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair of phrases, and compute the composition of its vectors using each of these functions. Thus, you will have (at least) three compositional models that take each `noun - verb` phrase from the pair (these phrases can be either references or landmarks) and output a single vector, representing the meaning of this phrase. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper.

**Thoughts process behind calculating the correlation:**

Let's look at the example pair: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. Let's say we have 3 humans evaluating the similarity between these two phrases and we combine their scores into a single vector: `[5, 6, 5]`. We need to average them to get our human vector for correlation: `[5.3]`.

Our A model's output:  
`cosine(p1, p2) = 0.88`, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector `[5.3]` and model A output `[0.88]`. Next is to compute correlation between these two vectors. This should give you a correlation value and p-value for the model of choice and human ratings.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]). Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

---

**The minimum to do in this task**: compute correlations for at least _ONE_ model and human ratings. However, this should not be hard to run it for any other model as well. For examples on how to interpret the results, look at Section 5 Results of the original paper.

In [20]:
import pandas as pd 

In [22]:
filename = "mitchell_lapata_acl08.txt"

In [23]:
class ProbeModels:
    def __init__(self, filename, space, model, 
                 alpha = 0, beta = 0.95, gamma = 0.05,
                 weight_n = 0.2, weight_v=0.8):
        self.df = pd.read_csv(filename, sep=" ")
        self.space = space
        self.model = model
        self.alpha = alpha 
        self.beta = beta
        self.gamma = gamma
        
        self.weight_n = weight_n
        self.weight_v = weight_v
        
    def combine_vectors(self, vector1, vector2):
        if self.model == "multiply":
            return vector1 * vector2
        elif self.model == "add":
            return vector1 + vector2
        elif self.model == "weighted_add":
            return self.weight_n * vector1 + self.weight_v * vector2
        elif self.model == "combine":
            return self.alpha * vector1 + self.beta * vector2 + self.gamma * vector1 * vector2
        else:
            raise NotImplementedError(f"{self.model} is not implemented")
    
    def get_similarities(self, vec1, vec2):
        veclen1 = veclen(vec1)
        veclen2 = veclen(vec2)

        if veclen1 == 0.0 or veclen2 == 0.0:
            # one of the vectors is empty. make the cosine zero.
            return 0.0

        else:
            # we could also simply do:
            # dotproduct = numpy.dot(vec1, vec2)
            dotproduct = numpy.sum(vec1 * vec2)

            return dotproduct / (veclen1 * veclen2)
        
    def get_spearman(self, sims, human):
        rho, pval = stats.spearmanr(sims, human)
        return rho, pval
        
    def process_row(self, row):
        if row["noun"] in self.space and row["verb"] in self.space and row["landmark"] in self.space:
            noun_vector = self.space[row["noun"]]
            verb_vector = self.space[row["verb"]]
            land_vector = self.space[row["landmark"]]

            ref_vector = self.combine_vectors(noun_vector, verb_vector)
            land_vector = self.combine_vectors(noun_vector, land_vector)

            cos_sim = self.get_similarities(ref_vector, land_vector)
            return cos_sim.tolist()
        
    def get_human_score(self):
        unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
        high_mean = unique_triplets[unique_triplets["hilo"]=="high"]["input"].mean()
        low_mean = unique_triplets[unique_triplets["hilo"]=="low"]["input"].mean()
        return {"model": "human", "High": high_mean, "Low": low_mean, "R": 1.0, "p-value": 0.0}
    
    def probe(self):
        unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
        unique_triplets["sims"] = unique_triplets.apply(self.process_row, axis=1)
        unique_triplets = unique_triplets[unique_triplets.sims.notna()]
        r, p = self.get_spearman(unique_triplets["sims"], unique_triplets["input"])
        high_mean = unique_triplets[unique_triplets["hilo"]=="high"]["sims"].mean()
        low_mean = unique_triplets[unique_triplets["hilo"]=="low"]["sims"].mean()
        return {"model": self.model, "High": high_mean, "Low": low_mean, "R": r, "p-value": p}

In [24]:
def run_all_models(filename, space):
    multiply = ProbeModels(filename, space, "multiply")
    combined = ProbeModels(filename, space, "combine")
    add = ProbeModels(filename, space, "add")
    weighted_add = ProbeModels(filename, space, "weighted_add")
    
    scores = []
    scores.append(multiply.get_human_score())
    scores.append(multiply.probe())
    scores.append(add.probe())
    scores.append(weighted_add.probe())
    scores.append(combined.probe())
    return pd.DataFrame(scores)

In [25]:
print("Results for raw space")
run_all_models(filename, space_10k)

Results for raw space


/tmp/ipykernel_3436275/216900176.py:59: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
/tmp/ipykernel_3436275/216900176.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
/tmp/ipykernel_3436275/216900176.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be val

,model,High,Low,R,p-value
0,human,5.084540,3.286199,1.000000,0.000000
1,multiply,0.908243,0.895953,0.166667,0.693239
2,add,0.986482,0.964184,0.452381,0.260405
3,weighted_add,0.941778,0.861051,0.404762,0.319889
4,combine,0.902514,0.879121,0.285714,0.492726


In [26]:
print("Results for ppmi space")
run_all_models(filename, ppmispace_10k)

Results for ppmi space


/tmp/ipykernel_3436275/216900176.py:59: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
/tmp/ipykernel_3436275/216900176.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
/tmp/ipykernel_3436275/216900176.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be val

,model,High,Low,R,p-value
0,human,5.084540,3.286199,1.000000,0.000000
1,multiply,0.077045,0.035370,0.571429,0.138960
2,add,0.629155,0.558364,0.642857,0.085559
3,weighted_add,0.157083,0.130878,0.690476,0.057990
4,combine,0.053801,0.052853,0.380952,0.351813


In [27]:
print("Results for svd space")
run_all_models(filename, svdspace_10k)

Results for svd space


/tmp/ipykernel_3436275/216900176.py:59: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
/tmp/ipykernel_3436275/216900176.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_triplets = self.df.groupby(["noun", "verb", "landmark", "hilo"]).mean().reset_index()
/tmp/ipykernel_3436275/216900176.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be val

,model,High,Low,R,p-value
0,human,5.084540,3.286199,1.000000,0.000000
1,multiply,0.934981,0.931578,0.047619,0.910849
2,add,0.926754,0.876599,0.571429,0.138960
3,weighted_add,0.702953,0.644893,0.404762,0.319889
4,combine,0.251353,0.293039,-0.095238,0.822505


**Any comments/thoughts should go here:**

These results do not agree with the results from the paper. On the other hand, a lot of the example sentence pairs could not be included because they contain out of vocabulary words, so even if the results had agreed with those in the paper, they should still be taken with a (large) grain of salt. As we say in these cases: it is not our fault, it's the data.

Even so, it is interesting that the versions that showed the highest correlation in the paper showed a very low correlation here (even negative in the case of the combined model). 

# Literature

[1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

[2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
[3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

This assignment has a total of 60 marks.